In [5]:
import logging
import asyncio
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the admin user ID (initially empty)
ADMIN_USER_ID = "247169756"

# Store a mapping of message IDs to user IDs
message_user_map = {}

# Define the command /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text('Hi! Send me a message and I will redirect it to the admin.')

# Define the command /get_id
async def get_id(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_id = update.message.from_user.id
    await update.message.reply_text(f'Your user ID is {user_id}')
    # Optionally, you can also log this or set the ADMIN_USER_ID variable directly

# Define a function to handle incoming messages from users
async def handle_user_message(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user = update.message.from_user
    user_id = user.id
    user_message = update.message.text if update.message.text else ""
    username = f'@{user.username}' if user.username else user.full_name
    forwarded_message = f"Message from {username}:\n{user_message}"

    # Send text messages
    if update.message.text:
        sent_message = await context.bot.send_message(chat_id=ADMIN_USER_ID, text=forwarded_message)
    
    # Handle photos
    elif update.message.photo:
        photo = update.message.photo[-1].file_id
        caption = update.message.caption if update.message.caption else ""
        sent_message = await context.bot.send_photo(chat_id=ADMIN_USER_ID, photo=photo, caption=f"{forwarded_message}\n{caption}")
    
    # Handle videos
    elif update.message.video:
        video = update.message.video.file_id
        caption = update.message.caption if update.message.caption else ""
        sent_message = await context.bot.send_video(chat_id=ADMIN_USER_ID, video=video, caption=f"{forwarded_message}\n{caption}")
    
    # Handle audio
    elif update.message.audio:
        audio = update.message.audio.file_id
        sent_message = await context.bot.send_audio(chat_id=ADMIN_USER_ID, audio=audio, caption=forwarded_message)
    
    # Handle documents (including zip/rar files)
    elif update.message.document:
        document = update.message.document.file_id
        sent_message = await context.bot.send_document(chat_id=ADMIN_USER_ID, document=document, caption=forwarded_message)
    
    # Handle other file types if necessary
    else:
        await update.message.reply_text('Unsupported file type.')

    # Store the message-user mapping
    message_user_map[sent_message.message_id] = user_id

# Define a function to handle replies from the admin
async def handle_admin_reply(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    if update.message.reply_to_message and update.message.reply_to_message.message_id in message_user_map:
        original_sender_id = message_user_map[update.message.reply_to_message.message_id]
        reply_message = update.message.text if update.message.text else ""
        # Send the admin's reply to the original sender
        await context.bot.send_message(chat_id=original_sender_id, text=f"Reply from admin: {reply_message}")

        # Handle photos in admin replies
        if update.message.photo:
            photo = update.message.photo[-1].file_id
            await context.bot.send_photo(chat_id=original_sender_id, photo=photo, caption=f"Reply from admin:\n{reply_message}")
        
        # Handle videos in admin replies
        elif update.message.video:
            video = update.message.video.file_id
            await context.bot.send_video(chat_id=original_sender_id, video=video, caption=f"Reply from admin:\n{reply_message}")
        
        # Handle audio in admin replies
        elif update.message.audio:
            audio = update.message.audio.file_id
            await context.bot.send_audio(chat_id=original_sender_id, audio=audio, caption=f"Reply from admin:\n{reply_message}")
        
        # Handle documents in admin replies
        elif update.message.document:
            document = update.message.document.file_id
            await context.bot.send_document(chat_id=original_sender_id, document=document, caption=f"Reply from admin:\n{reply_message}")

async def main() -> None:
    # Create the Application and pass it your bot's token
    application = Application.builder().token("7097597603:AAGQRSVF4hfaOxgxCMUrZGDGITz9c90FXxE").build()

   # on different commands - answer in Telegram
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("get_id", get_id))

    # on noncommand i.e message - handle the message
    application.add_handler(MessageHandler(filters.ALL & ~filters.REPLY, handle_user_message))
    application.add_handler(MessageHandler(filters.REPLY, handle_admin_reply))

    # Check if an event loop is already running
    try:
        loop = asyncio.get_running_loop()
        logger.info("Event loop is already running")
        await application.start()
    except RuntimeError:
        logger.info("Starting new event loop")
        # Start the Bot
        application.run_polling()

if __name__ == '__main__':
    asyncio.run(main())


2024-06-23 13:13:16,213 - __main__ - INFO - Event loop is already running
